In [17]:
import pandas as pd
# Read Text_Ham_Spam_Calssification csv file
Df = pd.read_csv("Text_Ham_Spam_Calssification.csv",delimiter=",")

In [18]:
# Rows and columns in Dataframe
Df.shape

(5026, 2)

In [19]:
Df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
3,ham,U dun say so early hor... U c already then say...
4,ham,Your gonna have to pick up a $1 burger for yourself on your way home. I can't even move. Pain is...


In [20]:
Df.tail()

,Category,Message
5021,NaN,NaN
5022,NaN,NaN
5023,NaN,NaN
5024,NaN,NaN
5025,NaN,NaN


In [21]:

df1 = Df[0:1790]
df1.shape

(1790, 2)

In [22]:

df2 = Df[1790:]
df2.isnull().sum()


Category    3235
Message     3235
dtype: int64

In [23]:
# Out of 1790 messages, 65 messages got categories null(Missing Categories) and I am considering this as my test data to predict categories for these 65 messages.
df1.isnull().sum()

Category    65
Message      0
dtype: int64

In [26]:
# Test data
Missing_categories = df1[df1["Category"].isnull()]
Missing_categories.to_csv("Test_data.csv", index = False)

In [27]:
Missing_categories

,Category,Message
1000,NaN,I'm at work. Please call
1001,NaN,Then u drive lor.
1002,NaN,Ard 515 like dat. Y?
1003,NaN,Tell me they're female :V how're you throwing in? We're deciding what all to get now
1004,NaN,EASTENDERS TV Quiz. What FLOWER does DOT compare herself to? D= VIOLET E= TULIP F= LILY txt D E ...
...,...,...
1323,NaN,"Ya tel, wats ur problem.."
1324,NaN,No. 1 Nokia Tone 4 ur mob every week! Just txt NOK to 87021. 1st Tone FREE ! so get txtin now an...
1325,NaN,i dnt wnt to tlk wid u
1326,NaN,We spend our days waiting for the ideal path to appear in front of us.. But what we forget is.. ...


In [9]:
# Dataframe containing only those rows which contain category HAM or SPAM are not null
df1.drop(Missing_categories.index, inplace = True) 
df1

C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,Your gonna have to pick up a $1 burger for you...
...,...,...
1785,ham,Yeah. I got a list with only u and Joanna if I...
1786,ham,I am in your office na.
1787,ham,Are you comingdown later?
1788,ham,Super da:)good replacement for murali


In [10]:
# Converting messages to embeddings (Words to vectors) using Spacy
import numpy as np
import pandas as pd
import re
import spacy
import pickle
import time
from datetime import timedelta
pd.set_option('display.max_colwidth', 100)

# Pre-trained Model :: 'en_vectors_web_lg'
nlp = spacy.load('en_vectors_web_lg')

In [11]:
# Could see the data is imbalanced where ham as 80 percentage and spam as 13 percentage 
# We need to balance them for that I am using Smoting technique
print(df1['Category'].value_counts(normalize = True))



ham     0.863188
spam    0.136812
Name: Category, dtype: float64


In [12]:
#Cleaning the text by doing the following operations.

# Define a list of punctuation marks
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']

In [13]:
# Code to replace punctuation marks with whitespace 
def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, ' ')
    return x

# Remove URL's from df1 and Missing_categories datasets
df1['clean_Message'] = df1['Message'].apply(lambda x: re.sub(r'http\S+', '', x))
Missing_categories['clean_Message'] = Missing_categories['Message'].apply(lambda x: re.sub(r'http\S+', '', x))

# Remove user handles @
df1['clean_Message'] = df1['clean_Message'].apply(lambda x: re.sub("@[\w]*", '', x))
Missing_categories['clean_Message'] = Missing_categories['clean_Message'].apply(lambda x: re.sub("@[\w]*", '', x))

# Remove punctuation marks
df1['clean_Message'] = df1['clean_Message'].apply(lambda x: clean_text(x))
Missing_categories['clean_Message'] = Missing_categories['clean_Message'].apply(lambda x: clean_text(x))

# Convert text to lowercase
df1['clean_Message'] = df1['clean_Message'].str.lower()
Missing_categories['clean_Message'] = Missing_categories['clean_Message'].str.lower()

# Remove numbers
df1['clean_Message'] = df1['clean_Message'].str.replace("[0-9]", " ")
Missing_categories['clean_Message'] = Missing_categories['clean_Message'].str.replace("[0-9]", " ")

# Remove whitespaces
df1['clean_Message'] = df1['clean_Message'].apply(lambda x:' '.join(x.split()))
Missing_categories['clean_Message'] = Missing_categories['clean_Message'].apply(lambda x: ' '.join(x.split()))

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [13]:
# Analyzing the Original Message and Cleaned message
df1[249:253]

,Category,Message,clean_Message
249,spam,"SMS. ac Blind Date 4U!: Rodds1 is 21/m from Aberdeen, United Kingdom. Check Him out http://img. ...",sms ac blind date u rodds is m from aberdeen united kingdom check him out sms ac w icmb cktz r n...
250,ham,Yup... From what i remb... I think should be can book...,yup from what i remb i think should be can book
251,ham,Jos ask if u wana meet up?,jos ask if u wana meet up
252,ham,Lol yes. Our friendship is hanging on a thread cause u won't buy stuff.,lol yes our friendship is hanging on a thread cause u won t buy stuff


In [14]:
# Function to lemmatize the tokens to their basic forms to normalize the message


def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output


# Lemmatise the tokens
df1['clean_Message'] = lemmatization(df1['clean_Message'])
df1['clean_Message'] = df1['clean_Message'].str.replace("-PRON-", "")
Missing_categories['clean_Message'] = lemmatization(Missing_categories['clean_Message'])
Missing_categories['clean_Message'] = Missing_categories['clean_Message'].str.replace("-PRON-", "")

print(df1.tail())

print(Missing_categories.tail())
   



     Category  \
1785      ham   
1786      ham   
1787      ham   
1788      ham   
1789      ham   

                                                                          Message  \
1785  Yeah. I got a list with only u and Joanna if I'm feeling really anti social   
1786                                                      I am in your office na.   
1787                                                    Are you comingdown later?   
1788                                        Super da:)good replacement for murali   
1789                                     Da is good good player.why he is unsold.   

                                                                clean_Message  
1785  yes i get a list with only u and joanna if i be feel really anti social  
1786                                                   i be in your office na  
1787                                                   be you comingdown late  
1788                                     super da good replacement

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [15]:
# Convert cleaned tweets into Spacy word vectors
# The model returns 300-dimensional embeddings
tweets = df1['clean_Message']
word_vec = [nlp(word).vector for word in tweets]
X_tr = np.array(word_vec)

Missing_categories_tweets = Missing_categories['clean_Message']
Missing_categories_word_vec = [nlp(word).vector for word in Missing_categories_tweets]
X_te = np.array(Missing_categories_word_vec)

print(X_tr.shape, X_te.shape)


# Save Spacy_train_new as pickle file to resue later
pickle_out = open("Spacy_train.pickle","wb")
pickle.dump(X_tr, pickle_out)
pickle_out.close()


# Save Spacy_train_new as pickle file to resue later
pickle_out = open("Spacy_test.pickle","wb")
pickle.dump(X_te, pickle_out)
pickle_out.close()

(1725, 300) (65, 300)


In [16]:
# sklearn_ML_models_with_embeddings
# Load variation of word embeddings from Spacy.
pickle_in = open("Spacy_train.pickle","rb")      
X = pickle.load(pickle_in)
print(X.shape)

# Observe the distribution of positive and negative tweets
print(df1['Category'].value_counts())

# Assign targets to y variable
y = df1['Category']

(1725, 300)
ham     1489
spam     236
Name: Category, dtype: int64


In [17]:
# Split the training dataset into train and test subsets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
print(y_train.shape)
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
# We could see the imbalance in the target varaible , So to balance we are using Smoting technique below.
print(y_train.value_counts(normalize = True))

(1552,)
(1552, 300)
(173, 300)
(173,)
ham     0.86018
spam    0.13982
Name: Category, dtype: float64


In [18]:
# !pip install imblearn
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 42)
X_train_sm , y_train_sm = sm.fit_sample(X_train , y_train)

In [19]:
X_train_sm.shape
print(y_train_sm.value_counts(normalize = True))

spam    0.5
ham     0.5
Name: Category, dtype: float64


In [20]:
# Applying Random forest classifier for Ham Spam classification
from sklearn.ensemble import RandomForestClassifier

# we can do grid x=search to find out the best parameters , but my system is taking a lot of time to find those.
# So assuming the best parameters by myself as of now.


# from sklearn.model_selection import GridSearchCV
# # Create the parameter grid based on the results of random search 
# param_grid = {
#     'bootstrap': [True],
#     'max_depth': [6,10,25],
#     'max_features': [15,25,30],
#     'min_samples_leaf': [3, 4, 5],
#     'min_samples_split': [8, 12],
#     'n_estimators': [100, 150, 350]
# }
# # Create a based model
# rf = RandomForestClassifier()
# # Instantiate the grid search model
# grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
#                           cv = 3, n_jobs = -1, verbose = 2)
model = RandomForestClassifier(n_estimators=300,max_depth = 6,max_features=20)
model.fit(X_train_sm, y_train_sm)
predictions = model.predict(X_test)
print((predictions))

['ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'spam' 'spam' 'ham'
 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'spam' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'spam'
 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'spam'
 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham'
 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'h

In [21]:
# Import metrics to check the accuracy for the predictions made
from sklearn import metrics
# Print a confusion matrix
print(metrics.confusion_matrix(y_test,predictions))
# Print a classification report
print(metrics.classification_report(y_test,predictions))
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

[[152   2]
 [  0  19]]
              precision    recall  f1-score   support

         ham       1.00      0.99      0.99       154
        spam       0.90      1.00      0.95        19

    accuracy                           0.99       173
   macro avg       0.95      0.99      0.97       173
weighted avg       0.99      0.99      0.99       173

0.9884393063583815


In [ ]:
# As we could see above , we got 2 False Positives using Random forest calssifier after balancing the target varaible.
# Without Blancing the target varaibale I could see more False positives and Few False Negatives.  

In [22]:
# Loading corresponding test message embeddings as loaded for the training dataset
pickle_in = open("Spacy_test.pickle","rb")
test_X = pickle.load(pickle_in)
print(test_X.shape)

# Making predictions using trained model for the test dataset for final submission 
test_predictions = model.predict(test_X)
print(test_predictions)

(65, 300)
['ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'spam' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'spam'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'spam' 'ham' 'ham'
 'spam' 'ham' 'spam' 'ham' 'ham' 'ham']


In [23]:
#Finally filling the missing categories of 65 messages with spam and ham (Predictions)
Missing_categories["Category_ham_spam"] = test_predictions
final_predictions = Missing_categories[["Message","Category_ham_spam"]]
final_predictions.to_csv("final_predictions.csv",index = False)
final_predictions

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Message,Category_ham_spam
1000,I'm at work. Please call,ham
1001,Then u drive lor.,ham
1002,Ard 515 like dat. Y?,ham
1003,Tell me they're female :V how're you throwing in? We're deciding what all to get now,ham
1004,EASTENDERS TV Quiz. What FLOWER does DOT compare herself to? D= VIOLET E= TULIP F= LILY txt D E ...,spam
...,...,...
1323,"Ya tel, wats ur problem..",ham
1324,No. 1 Nokia Tone 4 ur mob every week! Just txt NOK to 87021. 1st Tone FREE ! so get txtin now an...,spam
1325,i dnt wnt to tlk wid u,ham
1326,We spend our days waiting for the ideal path to appear in front of us.. But what we forget is.. ...,ham


In [24]:
# Using SVM classifier 
######### Support Vector Machine #########3
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
## Create a pipeline so as to streamline the flow of the data into the ML model
text_clf = Pipeline([('clf', LinearSVC())]) #('tfidf', TfidfVectorizer()), ('scaler', MaxAbsScaler())

# Feed the training data through the pipeline
text_clf.fit(X_train_sm , y_train_sm )

# Make predictions
predictions = text_clf.predict(X_test)
print(predictions)


# Import metrics to check the accuracy and other features of the predictions made
from sklearn import metrics

# Print a confusion matrix
print(metrics.confusion_matrix(y_test,predictions))

# Print a classification report
print(metrics.classification_report(y_test,predictions))

# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

['ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'spam' 'ham' 'ham'
 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'spam' 'spam' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'spam'
 'spam' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'spam'
 'spam' 'ham' 'ham' 'spam' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'spam'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham'
 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham'
 'spam' 'ham' 'ham' 'ham' 'ham' 'spam' 'ham' 'ham'

In [ ]:
# As we could see above 5 False positives using Support Vector Machine .

In [25]:
#######classifying spam_ham using tfidf vectors#######

In [26]:
# Creating Vocabulary for tfidf vectorizer function
def flatten_words(list1d, get_unique=False):
    qa = [s.split() for s in list1d]
    if get_unique:
        return sorted(list(set([w for sent in qa for w in sent])))
    else:
        return [w for sent in qa for w in sent]
    

all_text = df1['clean_Message'].values.tolist() + Missing_categories['clean_Message'].values.tolist()
vocab = flatten_words(all_text, get_unique=True)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(2,3),vocabulary=vocab)
training_matrix = tfidf.fit_transform(df1['clean_Message'])
test_matrix = tfidf.fit_transform(Missing_categories['clean_Message'])
training  = pd.DataFrame(training_matrix.todense())
print(training.head(3))
print(training.shape)
test = pd.DataFrame(test_matrix.todense())
print(test.head(3))

   0     1     2     3     4     5     6     7     8     9     ...  3570  \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

   3571  3572  3573  3574  3575  3576  3577  3578  3579  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[3 rows x 3580 columns]
(1725, 3580)
   0     1     2     3     4     5     6     7     8     9     ...  3570  \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

   3571  3572  3573  3574  3575  3576  3577  3578  3579  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   

In [29]:
# Split the training dataset into train and test subsets
from sklearn.model_selection import train_test_split
y = df1['Category']
X_train, X_test, y_train, y_test = train_test_split(training, y, test_size=0.1, random_state=42)
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
## Create a pipeline so as to streamline the flow of the data into the ML model
text_clf = Pipeline([('clf', LinearSVC())]) #('tfidf', TfidfVectorizer()), ('scaler', MaxAbsScaler())

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)

# Make predictions
predictions = text_clf.predict(X_test)
# print(predictions)

# Import metrics to check the accuracy and other features of the predictions made
from sklearn import metrics

# Print a confusion matrix
print(metrics.confusion_matrix(y_test,predictions))


# Print a classification report
print(metrics.classification_report(y_test,predictions))

# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

[[154   0]
 [ 19   0]]
              precision    recall  f1-score   support

         ham       0.89      1.00      0.94       154
        spam       0.00      0.00      0.00        19

    accuracy                           0.89       173
   macro avg       0.45      0.50      0.47       173
weighted avg       0.79      0.89      0.84       173

0.8901734104046243


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
 # As we could see above using TFIDF for generating vectors and applying SVM on top of it.
# could see 19 False Negatives because deep semantic meaning and ordering of words is not considered in TFIDF vectorization.

In [ ]:
# For the given text dataset , using Spacy embeddings and applying Random forest on top of it gave fair results.